In [120]:
import gurobipy as gb
import numpy as np
import matplotlib.pyplot as plt

In [121]:
T = 20; n = 4
T = list(range(1,T+1)); G = list(range(n))

################ Adulyasak et al. 2014
# C = floor(2*sum(d)/len(T))
################ Archetti et al. 2011
# h0 = 8; f = 100*10*8; d = U[5, 25]

h = 8; c = 5*h; f = 2*15*h

# Results functions

In [122]:
def print_summary(m, T, G, f, h, d, y, x, z, l, w, I, ntabs = 0):

    print("\t"*ntabs+f"Objective: {round(m.getObjective().getValue(),2)}\tRuntime: {round(m.RunTime,5)}")
    print("\t"*ntabs+f"\tSetup cost: {round(sum(f*y[t].x for t in T),2)}")
    print("\t"*ntabs+f"\tHolding cost: {round(sum(h*I[t,g] for t in T for g in G),2)}\n")

    print("\t"*ntabs+" ", *[" "*(5-len(str(t)))+str(t) for t in T], "Total", sep="\t")
    print("\t"*ntabs+"-"*8*(3+len(T)))
    print("\t"*ntabs+"d", *[" "*(5-len(str(round(d[t]))))+str(round(d[t])) for t in T], sep="\t")
    print("\t"*ntabs+"-"*8*(3+len(T)))
    print("\t"*ntabs+"x", *[" "*(5-len(str(int(x[t]))))+str(int(x[t])) if int(x[t]) > 0 else "    ." for t in T], " "*(5-len(str(int(sum(x[t] for t in T)))))+str(int(sum(x[t] for t in T))), sep="\t")
    print("\t"*ntabs+"-"*8*(3+len(T)))

    for g in G:
        print("\t"*ntabs+f"z{g}", *[" "*(5-len(str(int(z[t,g]))))+str(int(z[t,g])) if int(z[t,g]) > 0 else "    ." for t in T],  " "*(5-len(str(int(sum(z[t,g] for t in T)))))+str(int(sum(z[t,g] for t in T))), sep="\t")
    print("\t"*ntabs+"-"*8*(3+len(T)))
    
    print("\t"*ntabs+"l", *[" "*(5-len(str(int(l[t]))))+str(int(l[t])) if int(l[t]) > 0 else "    ." for t in T], " "*(5-len(str(int(sum(l[t] for t in T)))))+str(int(sum(l[t] for t in T))), sep="\t")
    print("\t"*ntabs+"-"*8*(3+len(T)))

    for g in G[:-1]:
        print("\t"*ntabs+f"I{g}", *[" "*(5-len(str(int(I[t,g]))))+str(int(I[t,g])) if int(I[t,g]) > 0 else "    ." for t in T],  " "*(5-len(str(int(sum(I[t,g] for t in T)))))+str(int(sum(I[t,g] for t in T))), sep="\t")
    print("\t"*ntabs+"-"*8*(3+len(T)))

    print("\t"*ntabs+"w", *[" "*(5-len(str(int(w[t]))))+str(int(w[t])) if int(w[t]) > 0 else "    ." for t in T], " "*(5-len(str(int(sum(w[t] for t in T)))))+str(int(sum(w[t] for t in T))), sep="\t")

def check_models_objectives(it, T, G, f, h, x, I, ntabs = 0, s = "Transportation"):

    objs = [round(f*sum(1 for t in T if round(x[m][t],2) > 0) + h*sum(I[m][t,g] for t in T for g in G),2) for m in [0,1]]; resp = True
    if objs[0] != objs[1]: print("\t"*ntabs+f"{it} DIFFERENT OBJECTIVE VALUES - Standard: {objs[0]}, {s}: {objs[1]}"); resp = False

    return resp


# Models functions

In [137]:
def standard_model(T, G, n, h, c, f, C, I0, d, alpha, beta = 0, epsilon = 1, output = False, verbose = False, FIFO = 0):

    m = gb.Model("Standard Model")
    Tt = {t:[m for m in range(t,np.min((T[-1],t+n-1)) + 1)] for t in T}

    ######################################
    # DECISION VARIABLES
    ######################################

    # Whether there is production in period t \in T or not
    y = {t:m.addVar(name=f"y_{t}", vtype=gb.GRB.BINARY) for t in T}
    # Produced quantity in period t \in T
    x = {t:m.addVar(name=f"x_{t}", vtype=gb.GRB.CONTINUOUS) for t in T}
    # Amount of product of age g \in G that is used to fulfill demand in period t \in T
    z = {(t,g):m.addVar(name=f"z_{t,g}", vtype=gb.GRB.CONTINUOUS) for t in T for g in G}
    # Lost sales of period t \in T
    l = {t:m.addVar(name=f"l_{t}", vtype=gb.GRB.CONTINUOUS) for t in T}
    # Available inventory of product of age g \in G at the end of period t \in T
    I = {(t,g):m.addVar(name=f"I_{t,g}", vtype=gb.GRB.CONTINUOUS) for t in T for g in G}
    I.update({(0,g):I0[g] for g in I0})

    ######################################
    # CONSTRAINTS
    ######################################

    for t in T:
        # Inventory of fresh produce
        m.addConstr(I[t,0] == x[t] - z[t,0])

        for g in G[1:]: 
            # Inventory dynamics throughout the day   
            m.addConstr(I[t,g] == I[t-1,g-1] - z[t,g])
        
        # Production capacity
        m.addConstr(x[t] <= np.min((C,sum(d[m] for m in Tt[t])))*y[t])

        # Demand fulfillment and lost sales modeling
        m.addConstr(gb.quicksum(z[t,g] for g in G) + l[t] == d[t])

        # Individual demand service level constraint
        m.addConstr(gb.quicksum(z[t,k] for k in G)/d[t] >= beta)

    for g in G:
        # Total age service level constraint
        m.addConstr(gb.quicksum(z[t,k] for t in T for k in range(g+1)) >= alpha[g]*sum(d[t] for t in T))
    
    # Total waste control constraint
    m.addConstr(gb.quicksum(I[t,n-1] for t in T) <= epsilon*gb.quicksum(x[t] for t in T))

    if FIFO == 1:
        gamma = {(t,g):m.addVar(name=f"gamma_{t,g}", vtype=gb.GRB.BINARY) for t in T for g in G}

        for t in T:

            for g in G:
                m.addConstr(I[t,g] <= C*(1-gamma[t,g]))

                if g < G[-1]:
                    m.addConstr(gamma[t,g] <= gamma[t,g+1])
                    m.addConstr(z[t,g] <= C*gamma[t,g+1])
    
    elif FIFO == 2:
        r_pos = {(t,g):m.addVar(name=f"rpos_{t,g}", vtype=gb.GRB.CONTINUOUS) for t in T for g in G[1:]}
        r_neg = {(t,g):m.addVar(name=f"rneg_{t,g}", vtype=gb.GRB.CONTINUOUS) for t in T for g in G[1:]}

        for t in T:
            for g in G[1:]:

                m.addConstr(gb.quicksum(z[t,k] for k in G if k < g) == r_pos[t,g])
                m.addConstr(r_pos[t,g] - r_neg[t,g] == gb.quicksum(z[t,k] for k in G) - gb.quicksum(I[t-1,k-1] for k in G if k >= g))
    
    elif FIFO == 3:
        gamma = {(t,g):m.addVar(name=f"gamma_{t,g}", vtype=gb.GRB.BINARY) for t in T for g in G}

        for t in T:
            for g in G:
                
                if g > 0:
                    m.addConstr(gb.quicksum(z[t,k] for k in G if k >= g) >= gb.quicksum(I[t-1,k-1] for k in G if k >= g) - C*sum(1 for k in G if k >= g)*(1-gamma[t,g]))
                
                m.addConstr(gb.quicksum(z[t,k] for k in G if k >= g) >= gb.quicksum(z[t,k] for k in G) - d[t]*gamma[t,g])

                if g < G[-1]:
                    m.addConstr(gamma[t,g] <= gamma[t,g+1])

    
    ######################################
    # OBJECTIVE FUNCTION
    ######################################
            
    m.setObjective(gb.quicksum(f*y[t] + c*x[t] + h*gb.quicksum(I[t,g] for g in G) for t in T))

    m.update()
    m.setParam("OutputFlag",output)
    m.optimize()

    ######################################
    # RESULTS RETRIEVING
    ######################################

    x = {t:x[t].x for t in T}
    z = {(t,g):z[t,g].x for t in T for g in G}
    l = {t:l[t].x for t in T}
    w = {t:I[t,n-1].x for t in T}
    I = {(t,g):I[t,g].x for t in T for g in G}

    if verbose: print_summary(m, T, G, f, h, d, y, x, z, l, w, I)

    return x, z, l, w, I
    
def transportation_model(T, G, n, h, f, C, I0, d, alpha, beta = 0, epsilon = 1, feasibility = False, output = False, verbose = False):

    mm = gb.Model("Standard Model")
    Tt = {t:[m for m in range(t,np.min((T[-1],t+n-1)) + 1)] for t in T}

    ######################################
    # DECISION VARIABLES
    ######################################

    # Whether there is production in time period t \in T or not
    y = {t:mm.addVar(name=f"y_{t}",vtype=gb.GRB.BINARY) for t in T}
    # Production quantity of time period t \in \T
    x = {(t,m):mm.addVar(name=f"x_{t,m}",vtype=gb.GRB.CONTINUOUS) for t in T for m in Tt[t]}
    # Amount of initial inventory of age g \in G - {n-1} used to fulfill demand in time period t \in {1, ..., n-g-1}
    r = {(t,g):mm.addVar(name=f"r_{t,g}",vtype=gb.GRB.CONTINUOUS) for g in G[:-1] for t in range(1,n-g)}
    # Available inventory of age g \in \G at the end of time period t \in T
    I = {(t,g):mm.addVar(name=f"I_{t,g}",vtype=gb.GRB.CONTINUOUS) for t in T for g in G}
    # Lost sales of time period t \in T
    l = {t:mm.addVar(name=f"l_{t}",vtype=gb.GRB.CONTINUOUS) for t in T}
    # Generated waste in time period t \in T
    w = {t:mm.addVar(name=f"w_{t}",vtype=gb.GRB.CONTINUOUS) for t in T}

    ######################################
    # CONSTRAINTS
    ######################################      
    
    for g in G[:-1]:
        # Initial inventory modeling
        mm.addConstr(I0[g] == gb.quicksum(r[t,g] for t in range(1, n-g-1 + 1)) + w[n-g-1])
    
    for t in T:
        for g in G[:-1]:
            # Inventory of time periods and ages for which initial inventory can be used
            if t-g <= 0:
                mm.addConstr(I[t,g] == gb.quicksum(r[j,g-t] for j in range(t+1, (n-1) - (g-t) + 1)) + w[(n-1)-(g-t)])
            # Inventory of time periods and ages for which waste is realized within the planning horizon
            elif 1 <= t-g and t-g <= T[-1]-n+1:
                mm.addConstr(I[t,g] == gb.quicksum(x[t-g, m] for m in Tt[t-g] if m>t) + w[(n-1) + (t-g)])
            # Inventory for time periods and ages for which waste is NOT realized within the planning horizon
            elif t-g >= T[-1]-n+2 and t < T[-1]:
                mm.addConstr(I[t,g] == gb.quicksum(x[t-g, m] for m in Tt[t-g] if m>t))
        # Inventory of oldest product
        mm.addConstr(I[t, n-1] == w[t])

        # Demand modeling
        if t <= n-1:
            mm.addConstr(gb.quicksum(r[t,g] for g in range(n-t-1 + 1)) + gb.quicksum(x[j,t] for j in range(1, t+1)) + l[t] == d[t])
        else:
            mm.addConstr(gb.quicksum(x[j,t] for j in range(t-n+1, t+1)) + l[t] == d[t])
        
        # Production capacity constraints
        if t+n-1 <= T[-1]:
            mm.addConstr(gb.quicksum(x[t,m] for m in Tt[t]) + w[t+n-1] <= C*y[t])
        else:
            mm.addConstr(gb.quicksum(x[t,m] for m in Tt[t]) <= C*y[t])
        for m in Tt[t]:
            mm.addConstr(x[t,m] <= np.min((C,d[m]))*y[t])

    # Age service level constraints
    mm.addConstr(gb.quicksum(x[t,t] for t in T) >= alpha[0]*sum(d[t] for t in T))
    for g in G[1:]:
        mm.addConstr(gb.quicksum(x[t,m] for t in T for m in Tt[t] if m-t<=g) + gb.quicksum(r[t,k] for t in range(1, g+1) for k in range(g-t+1)) >= alpha[g]*sum(d[t] for t in T))
    
    # Demand service level per time period
    for t in T:
        if t <= n-1:
            mm.addConstr(gb.quicksum(r[t,g] for g in range(n-t-1 + 1)) + gb.quicksum(x[j,t] for j in range(1, t+1)) >= beta*d[t])
        else:
            mm.addConstr(gb.quicksum(x[j,t] for j in range(t-n+1, t+1)) >= beta*d[t])

    # Waste control constraint
    mm.addConstr(gb.quicksum(w[t] for t in T) <= epsilon*(gb.quicksum(x[t,m] for t in T for m in Tt[t])+gb.quicksum(w[t] for t in T if t >= n)))
    
    ######################################
    # OBJECTIVE FUNCTION
    ######################################
            
    mm.setObjective(gb.quicksum(f*y[t] + h*gb.quicksum(I[t,g] for g in G) for t in T))

    mm.update()
    mm.setParam("OutputFlag",output)
    mm.optimize()

    ######################################
    # RESULTS RETRIEVING
    ######################################

    xx = {t:sum(x[t,m].x for m in Tt[t])+w[t+n-1].x if t+n-1 <= T[-1] else sum(x[t,m].x for m in Tt[t]) for t in T}
    z = {(t,g):x[t-g,t].x if t-g >= 1 else r[t,g-t].x for t in T for g in G}
    l = {t:l[t].x for t in T}
    w = {t:w[t].x for t in T}
    I = {(t,g):I[t,g].x for t in T for g in G}

    ######################################
    # FEASIBILITY CHECK
    ######################################
    if feasibility:
        for t in T:
            if round(xx[t],3) > C*y[t].x: print(f"\t\tProduction capacity constraint for day {t} is violated. {xx[t]} > {C*y[t].x}")
            dem = round(sum(z[t,g] for g in G) + l[t],3)
            if dem != round(d[t],3): print(f"\t\tDemand modeling of period {t} doesn't add up. {dem} != {d[t]}")
        
        for g in G:
            sl = round(sum(z[t,k] for t in T for k in range(0, g+1))/sum(d[t] for t in T),3)
            if sl < alpha[g]: print(f"\t\tTotal age service level constraint for age {g} is violated. {sl} < {alpha[g]}")
        
        waste = sum(I[t, n-1] for t in T)/sum(xx[t] for t in T)
        if waste > epsilon: print(f"\t\tTotal waste level constraint is violated. {waste} > {epsilon}")

    if verbose: print_summary(mm, T, G, f, h, d, y, xx, l, w, I)

    return xx, z, l, w, I

# Other

In [138]:
verbose = True; reps = 1; beta = 0
I0 = {g:0 for g in G[:-1]}

for sl in range(7,11):

    print(f"\nChecking for total service level of {sl/10:.0%}")
    alpha = {g:0 if g < G[-1] else sl/10 for g in G}

    for aa in range(reps):
        d = {t:np.random.rand()*20+5 for t in T}; C = np.floor(2*sum(list(d.values()))/len(T))
        
        x1, z1, l1, w1, I1 = standard_model(T, G, n, h, c, f, C, I0, d, alpha, FIFO = 1, beta=beta, verbose=verbose)
        x2, z2, l2, w2, I2 = standard_model(T, G, n, h, c, f, C, I0, d, alpha, FIFO = 2, beta=beta, verbose=verbose)
        x3, z3, l3, w3, I3 = standard_model(T, G, n, h, c, f, C, I0, d, alpha, FIFO = 3, beta=beta, verbose=verbose)
        
        #resp = check_models_objectives(aa, T, G, f, h, (x1, x2), (I1, I2), ntabs=1, s="FIFO")




Checking for total service level of 70%
Objective: 10785.47	Runtime: 0.061
	Setup cost: 1920.0
	Holding cost: 351.06

 	    1	    2	    3	    4	    5	    6	    7	    8	    9	   10	   11	   12	   13	   14	   15	   16	   17	   18	   19	   20	Total
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
d	    8	   23	   13	   17	   24	   13	   20	   18	   12	   21	    5	   19	   12	    7	   22	   22	   15	    8	   17	    9
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
x	    .	   30	    .	    .	   30	    .	   20	   30	    .	   26	    .	   24	    .	    .	   21	   30	    .	    .	    .	    .	  212
-------------------------------------------------------------------------------------------------------------------------------------

# Formulations equivalence check

In [5]:
# [5,10,15,20,25], [1,2,3,4,5,10]

for tt in [10,20,30]:
    for n in [5,10]:
        
        if tt >= n and n <= tt/2:
            
            T = list(range(1,tt+1)); G = list(range(n))
            I0 = {g:10 for g in G[:-1]}
            print(f"Checking for |T| = {tt}, n = {n}")

            for a0 in [0, 0.2, 0.4, 0.6]:
                for a1 in [0.8, 0.9, 1]:

                    a = [round(a0+(a1-a0)*i/(n-1),3) if len(G)>1 else 0.7 for i in range(len(G))]
                    alpha = dict(zip(G,a))

                    print(f"\talpha_0 = {a0}, alpha_n1 = {a1}")

                    correct = 0
                    while correct <= 1000:
                        
                        #try:
                        d = {t:np.random.rand()*20+5 for t in T}
                        C = np.floor(2*sum(list(d.values()))/len(T))

                        x1, z1, l1, w1, I1 = standard_model(T, G, n, h, f, C, I0, d, alpha, FIFO = False)
                        x2, z2, l2, w2, I2 = transportation_model(T, G, n, h, f, C, I0, d, alpha)
                        resp = check_models_objectives(correct, T, G, f, h, (x1,x2), (I1,I2), ntabs=2)

                        if not resp: x2, z2, l2, w2, I2 = transportation_model(T, G, n, h, f, C, I0, d, alpha, feasibility = True)

                        correct += 1
                        if correct % 200 == 0: print(f"\t\t --- DONE {correct}")

                        #except:
                        #    continue

Checking for |T| = 10, n = 5
	alpha_0 = 0, alpha_n1 = 0.8
Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-14
		 --- DONE 200
		 --- DONE 400
		 --- DONE 600
		 --- DONE 800
		 --- DONE 1000
	alpha_0 = 0, alpha_n1 = 0.9
		 --- DONE 200
		205 DIFFERENT OBJECTIVE VALUES - Standard: 1523.09, Transportation: 1523.2
		 --- DONE 400
		 --- DONE 600
		 --- DONE 800
		 --- DONE 1000
	alpha_0 = 0, alpha_n1 = 1
		 --- DONE 200
		 --- DONE 400
		 --- DONE 600
		 --- DONE 800
		927 DIFFERENT OBJECTIVE VALUES - Standard: 1744.92, Transportation: 1745.0
		 --- DONE 1000
	alpha_0 = 0.2, alpha_n1 = 0.8
		 --- DONE 200
		 --- DONE 400
		 --- DONE 600
		 --- DONE 800
		 --- DONE 1000
	alpha_0 = 0.2, alpha_n1 = 0.9
		 --- DONE 200
		376 DIFFERENT OBJECTIVE VALUES - Standard: 1634.08, Transportation: 1634.15
		 --- DONE 400
		 --- DONE 600
		 --- DONE 800
		 --- DONE 1000
	alpha_0 = 0.2, alpha_n1 = 1
		 --- DONE 200
		 --- DONE 400
		 --- DONE 600
		 --- DONE 800
		 